In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
coef_path = models_dir_path / "coef.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= "2014-01") & (train["date"] < validation_start_date)
is_valid = train["date"] >= validation_start_date

In [ ]:
is_train |= is_valid

In [ ]:
train = train[is_train]

In [ ]:
grouped = train.groupby([pd.Grouper(key="date", freq="Y"), "store_id"])
y = grouped[target].mean()
y = y.unstack()

n_samples, _ = y.shape
X = np.arange(n_samples)
X = X.reshape(-1, 1)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X, y)

In [ ]:
coef = 1.0 + model.coef_
coef = coef.ravel()
coef = pd.Series(coef, name="coef", index=y.columns)
coef = coef.astype("float32")

In [ ]:
coef.plot(kind="bar", figsize=(16, 9), grid=True)

In [ ]:
joblib.dump(coef, coef_path)